In [ ]:
import torch
from pssr.data import ImageDataset, SlidingDataset, PairedImageDataset, PairedSlidingDataset
from pssr.crappifiers import AdditiveGaussian, Poisson, SaltPepper, MultiCrappifier
from pssr.models import ResUNet, RDResUNet, SwinIR
from pssr.predict import predict_images

# Using low-resolution images

In [ ]:
test_data_path = "your/path" # Folder with low-resolution images to be upscaled
model_path = "your/path" # Path to pretrained model

# Set to same values as training notebook, ImageDataset infers that low-resolution data is being loaded
# (If using SlidingDataset to load low-resolution images, instead set hr_res to hr_res/lr_scale and lr_scale to -1,
# correct scale must still be specified to model)
hr_res = 512
lr_scale = 4
n_frames = -1

# Assuming the folder contains only testing images, we want to use all images
val_split = 1

In [ ]:
test_dataset = ImageDataset(test_data_path, hr_res, lr_scale, n_frames, val_split=val_split)

In [ ]:
# Set same arguments as pretrained model
model = ResUNet(
    channels=1 if n_frames == -1 else n_frames,
    scale=lr_scale,
)
model.load_state_dict(torch.load(model_path))

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Upscaled images will be saved to "preds" folder
predict_images(model, test_dataset, device, out_dir="preds")

# Using paired high-low-resolution images for benchmarking

In [ ]:
# Comment to run the rest of the notebook
assert False

In [ ]:
from pssr.predict import predict_collage, test_metrics

In [ ]:
# Set to paths of aligned high-resolution and low-resolution images
test_hr_path, test_lr_path = "your/path", "your/path"

paired_dataset = PairedImageDataset(test_hr_path, test_lr_path, hr_res, lr_scale, n_frames)

In [ ]:
# Normalizes image intensities for fair benchmarking, disable if strange results occur
norm = True

In [ ]:
predict_collage(model, paired_dataset, device, norm=norm, prefix="test", out_dir="preds")
test_metrics(model, test_dataset, device, norm=norm)